In [155]:
import pandas as pd
import numpy as np

In [2]:
Lab = pd.read_csv("FullData/LabReduced.csv", usecols = ['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc', 'UnitOfMeasure_calc'])
Exam = pd.read_csv("FullData/Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])
DiseaseCase = pd.read_csv("FullData/DiseaseCase.csv")

/Users/davidhuang/miniforge3/envs/tfARM/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# DiseaseCase
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])

In [4]:
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))

sBP (mmHg)                  2570192
dBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64


In [5]:
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
Lab = Lab.loc[Lab.Name_calc.isin(tests)]
Exam = Exam.loc[Exam.Name_calc.isin(tests)]

In [6]:
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]
Exam = Exam.drop_duplicates(subset=['Patient_ID','PerformedDate'])

In [7]:
lab = pd.concat([Lab,Exam])

In [8]:
lab.isna().sum()

Patient_ID                 0
PerformedDate         432244
Name_calc                  0
TestResult_calc            0
UnitOfMeasure_calc     28971
dtype: int64

In [9]:
lab = lab.dropna(subset = ['PerformedDate'])
lab.isna().sum()

Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc    28660
dtype: int64

In [10]:
disease_periods = pd.read_csv('AP_posnegIDS.csv', index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)
neg_sample = disease_periods.iloc[1902:3804]
disease_periods = disease_periods.iloc[0:1902]

In [11]:
disease_periods.head()

,Patient_ID,DateOfOnset,CompOnset
0,1000100165845,2006-02-14,2013-03-04
1,1000100082336,2004-10-18,2008-04-08
2,1000100035828,2004-03-09,2009-09-28
3,1000100033944,2005-06-17,2007-03-16
4,1000100166298,2013-01-30,2017-01-18


In [12]:
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')
print(len(patient_records.index),patient_records.columns)

243260 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'TestResult_calc',
       'UnitOfMeasure_calc', 'DateOfOnset', 'CompOnset'],
      dtype='object')


In [13]:
patient_records.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset,CompOnset
0,1000100166298,2016-04-23,Leukocytes in Blood (WBC),8.7,10^9/L,2013-01-30,2017-01-18
1,1000100166298,2017-10-19,Total Cholesterol (TCh),2.25,mmol/L,2013-01-30,2017-01-18
2,1000100166298,2019-07-26,Erythrocytes in Blood (RBC),5.5,10^12/L,2013-01-30,2017-01-18
3,1000100166298,2018-12-12,Platelets in Blood (PLT),352,10^9/L,2013-01-30,2017-01-18
4,1000100166298,2018-08-15,Platelets in Blood (PLT),302,10^9/L,2013-01-30,2017-01-18


In [70]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]
len(relVisits.index)

77217

In [72]:
# Count number of visits per patient
df = relVisits[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() < 51
good = df.index[df['PerformedDate'] == True].tolist()
n = len(good)
print(n)
relVisits = relVisits[relVisits['Patient_ID'].isin(good)]

1688


In [73]:
relVisits.PerformedDate = pd.to_datetime(relVisits.PerformedDate,format='%Y-%m-%d')
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [74]:
print(relVisits.isna().sum())
print(relVisits.dtypes)

Patient_ID            0
PerformedDate         0
Name_calc             0
TestResult_calc       0
UnitOfMeasure_calc    0
DateOfOnset           0
CompOnset             0
dtype: int64
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
TestResult_calc               object
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
CompOnset             datetime64[ns]
dtype: object


In [75]:
# Remove patients with < 5 results
#print(Results.groupby('Patient_ID').agg('count'))
n= relVisits['Patient_ID'].unique()
print(len(n))
print(len(relVisits.index) / len(relVisits['Patient_ID'].unique())) # Average visits per person
#len(a.index)

1688
40.37677725118483


In [76]:
relVisits['dp'] = relVisits['CompOnset'] - relVisits['DateOfOnset']
dp = relVisits['dp'].mean()
print(dp)

1961 days 05:34:07.091965504


In [79]:
# Negative sample
from random import sample
rest = Lab[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() < 51
negGood = rest.index[rest['PerformedDate'] == True].tolist()
print(len(negGood))
neg_ids = list(set(negGood) - set(n))
neg_samp = sample(neg_ids, len(n))
print(len(neg_samp))
neg_rec = lab.loc[lab['Patient_ID'].isin(neg_samp)]
neg_rec = pd.merge(neg_rec, DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[neg_rec['PerformedDate'] > neg_rec['DateOfOnset']]
print(len(neg_rec.index)/ len(neg_samp))
neg_rec = neg_rec[(neg_rec['PerformedDate'] > neg_rec['DateOfOnset']) & (neg_rec['PerformedDate'] < neg_rec['DateOfOnset']+dp) ]
print(len(neg_rec.index)/ len(neg_samp))

135497
1688
71.96741706161137
56.13744075829384
38.01599526066351


In [80]:
print(len(relVisits.index))
print(len(neg_rec.index))
relVisits = relVisits.append(neg_rec, ignore_index=True)
print(len(relVisits.index))
print(relVisits.isna().sum())
print(relVisits.dtypes)

68156
64171
132327
Patient_ID                0
PerformedDate             0
Name_calc                 0
TestResult_calc           0
UnitOfMeasure_calc       75
DateOfOnset               0
CompOnset             64171
dp                    64171
dtype: int64
Patient_ID                      int64
PerformedDate                  object
Name_calc                      object
TestResult_calc                object
UnitOfMeasure_calc             object
DateOfOnset            datetime64[ns]
CompOnset              datetime64[ns]
dp                    timedelta64[ns]
dtype: object


In [81]:
combined = relVisits.drop(['CompOnset','dp'],axis='columns')
print(combined.isna().sum())
print(combined.dtypes)
combined.TestResult_calc = pd.to_numeric(combined.TestResult_calc,errors='coerce')
combined = combined.dropna()
print(len(combined.index))

Patient_ID             0
PerformedDate          0
Name_calc              0
TestResult_calc        0
UnitOfMeasure_calc    75
DateOfOnset            0
dtype: int64
Patient_ID                     int64
PerformedDate                 object
Name_calc                     object
TestResult_calc               object
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
dtype: object
130715


In [82]:
#remove vast minority units (mg/L for Total Bilirubin (n = 40) and mmol/L for Hemoglobin (n = 10))
badIndices = combined.loc[(combined.Name_calc == 'Total Bilirubin (TBIL)') & (relVisits.UnitOfMeasure_calc == 'mg/L')].index
badIndices2 = combined.loc[(combined.Name_calc == 'Hemoglobin (Hb)') & (relVisits.UnitOfMeasure_calc == 'mmol/L')].index
combined = combined.drop(badIndices)
combined = combined.drop(badIndices2)

In [113]:
Results = combined
print(len(Results.index))

130715


In [114]:
for test in tests:
    q1 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = Results.loc[Results.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = Results.loc[(Results.Name_calc == test) & ((Results.TestResult_calc < (q1 - rng)) 
                                    |(Results.TestResult_calc > (q3 + rng)))].index
    Results = Results.drop(badIndices)

In [115]:
print(len(Results.index))
Results.head()

121363


,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100166298,2016-04-23 00:00:00,Leukocytes in Blood (WBC),8.7,10^9/L,2013-01-30
1,1000100166298,2014-04-08 00:00:00,Hemoglobin A1c (HbA1c),9.2,%,2013-01-30
2,1000100166298,2015-09-29 00:00:00,Hemoglobin A1c (HbA1c),7.5,%,2013-01-30
3,1000100166298,2014-10-31 00:00:00,Hemoglobin A1c (HbA1c),7.9,%,2013-01-30
4,1000100166298,2014-07-23 00:00:00,Hemoglobin A1c (HbA1c),8.8,%,2013-01-30


In [116]:
turtiles = pd.read_csv('TestTurtiles.csv')

In [117]:
turtiles

,Unnamed: 0,TestName,t1,t2
0,0,Alanine Aminotransferase in Serum or Plasma (ALT),18.00,27.00
1,1,Aspartate Aminotransferase in Serum or Plasma ...,18.00,24.00
2,2,Urea (mM),5.10,6.70
3,3,Plasma Creatinine Clearance (eGFR),60.00,83.00
4,4,Potassium (mM),4.20,4.60
5,5,Sodium (mM),139.00,141.00
6,6,Gamma Glutamyl Transferase in Serum or Plasma ...,23.00,41.00
7,7,Leukocytes in Blood (WBC),6.20,7.90
8,8,"Protein, total (g/L)",63.00,71.00
9,9,Total Cholesterol (TCh),3.53,4.40


In [118]:
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    #feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    #print(t1,t2,value)
    if value < t1 :
        feature = 1.0 #[1,0,0]
    elif value > t2 :
        feature = 3.0 #[0,0,1]
    else :
        feature = 2.0 #[0,1,0]
    Results.at[index, 'TestResult_calc'] = feature
    
        

In [119]:
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100166298,2016-04-23 00:00:00,Leukocytes in Blood (WBC),3.0,10^9/L,2013-01-30
1,1000100166298,2014-04-08 00:00:00,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
2,1000100166298,2015-09-29 00:00:00,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
3,1000100166298,2014-10-31 00:00:00,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30
4,1000100166298,2014-07-23 00:00:00,Hemoglobin A1c (HbA1c),3.0,%,2013-01-30


In [120]:
Results.groupby('TestResult_calc').agg('count')

,Patient_ID,PerformedDate,Name_calc,UnitOfMeasure_calc,DateOfOnset
TestResult_calc,,,,,
1.0,37725,37725,37725,37725,37725
2.0,44303,44303,44303,44303,44303
3.0,39335,39335,39335,39335,39335


In [104]:
"""Results.PerformedDate = pd.to_datetime(Results.PerformedDate,format='%Y-%m-%d')
Results = Results.sort_values(by='PerformedDate', ascending = True)
Results.head()"""

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
98878,8080570037419,2001-02-21,Sodium (mM),2.0,mmol/L,2000-09-28
98864,8080570037419,2001-02-21,Potassium (mM),2.0,mmol/L,2000-09-28
98877,8080570037419,2001-02-21,Alanine Aminotransferase in Serum or Plasma (ALT),3.0,U/L,2000-09-28
98875,8080570037419,2001-02-21,Hemoglobin (Hb),3.0,g/L,2000-09-28
98876,8080570037419,2001-02-27,Total Cholesterol (TCh),3.0,mmol/L,2000-09-28


In [125]:
Results = Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
Results = Results.reset_index(drop = True)
Results.head()

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100000266,2006-02-07 00:00:00,Plasma Creatinine Clearance (eGFR),1.0,mL/min,2006-01-31
1,1000100000266,2006-02-07 00:00:00,Total Cholesterol (TCh),1.0,mmol/L,2006-01-31
2,1000100000266,2006-02-07 00:00:00,Fasting Blood Glucose (FBG),1.0,mmol/L,2006-01-31
3,1000100000266,2006-02-07 00:00:00,Alanine Aminotransferase in Serum or Plasma (ALT),3.0,U/L,2006-01-31
4,1000100000266,2006-02-07 00:00:00,Gamma Glutamyl Transferase in Serum or Plasma ...,2.0,U/L,2006-01-31


In [126]:
rem = []
for index, row in Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))


5930


In [127]:
Results_u = Results
print(len(Results_u.index))
Results_u = Results_u.drop(rem)
print(len(Results_u.index))

121363
115433


In [128]:
Results_u.head(40)

,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100000266,2006-02-07 00:00:00,Plasma Creatinine Clearance (eGFR),1.0,mL/min,2006-01-31
1,1000100000266,2006-02-07 00:00:00,Total Cholesterol (TCh),1.0,mmol/L,2006-01-31
2,1000100000266,2006-02-07 00:00:00,Fasting Blood Glucose (FBG),1.0,mmol/L,2006-01-31
3,1000100000266,2006-02-07 00:00:00,Alanine Aminotransferase in Serum or Plasma (ALT),3.0,U/L,2006-01-31
4,1000100000266,2006-02-07 00:00:00,Gamma Glutamyl Transferase in Serum or Plasma ...,2.0,U/L,2006-01-31
5,1000100000266,2006-02-07 00:00:00,Leukocytes in Blood (WBC),2.0,10^9/L,2006-01-31
6,1000100000266,2006-02-07 00:00:00,Erythrocytes in Blood (RBC),3.0,10^12/L,2006-01-31
7,1000100000266,2006-02-07 00:00:00,Hemoglobin (Hb),3.0,g/L,2006-01-31
8,1000100000266,2006-02-07 00:00:00,Platelets in Blood (PLT),1.0,10^9/L,2006-01-31
9,1000100000266,2006-02-07 00:00:00,Hemoglobin A1c (HbA1c),1.0,%,2006-01-31


In [131]:
AllVisits = {}
VisitValues = {}
for index, row in Results_u.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    diag = (row['Name_calc'],row['TestResult_calc'])
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    AllVisits[temp] = AllVisits[temp] + [diag]

PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
PVDF = PVDF.transpose()
PVDF = PVDF.dropna(axis=0,how='all') # Remove NA


In [132]:
PVDF

0   \
1000100000266  2006-02-07 00:00:00          (Plasma Creatinine Clearance (eGFR), 1.0)   
               2006-02-13 00:00:00                                (BMI (kg/m^2), 2.0)   
               2006-05-11 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
1000100000369  2013-05-02                               (Hemoglobin A1c (HbA1c), 1.0)   
               2013-12-30                              (Total Cholesterol (TCh), 3.0)   
...                                                                               ...   
12120120066987 2017-07-11 00:00:00                             (Hemoglobin (Hb), 1.0)   
               2017-09-25 00:00:00                                (BMI (kg/m^2), 1.0)   
               2017-10-02 00:00:00                             (Hemoglobin (Hb), 1.0)   
               2018-02-02 00:00:00                      (Hemoglobin A1c (HbA1c), 2.0)   
               2018-03-20 00:00:00                             (Hemoglobin (Hb), 1.0)   

                                                                    1   \
1000100000266  2006-02-07 00:00:00      (Total Cholesterol (TCh), 1.0)   
               2006-02-13 00:00:00                                 NaN   
               2006-05-11 00:00:00       (Hemoglobin A1c (HbA1c), 1.0)   
1000100000369  2013-05-02                                          NaN   
               2013-12-30           (Fasting Blood Glucose (FBG), 1.0)   
...                                                                ...   
12120120066987 2017-07-11 00:00:00                                 NaN   
               2017-09-25 00:00:00                                 NaN   
               2017-10-02 00:00:00                                 NaN   
               2018-02-02 00:00:00                                 NaN   
               2018-03-20 00:00:00                                 NaN   

                                                                           2   \
1000100000266  2006-02-07 00:00:00         (Fasting Blood Glucose (FBG), 1.0)   
               2006-02-13 00:00:00                                        NaN   
               2006-05-11 00:00:00  (Plasma Creatinine Clearance (eGFR), 1.0)   
1000100000369  2013-05-02                                                 NaN   
               2013-12-30                                                 NaN   
...                                                                       ...   
12120120066987 2017-07-11 00:00:00                                        NaN   
               2017-09-25 00:00:00                                        NaN   
               2017-10-02 00:00:00                                        NaN   
               2018-02-02 00:00:00                                        NaN   
               2018-03-20 00:00:00                                        NaN   

                                                                                   3   \
1000100000266  2006-02-07 00:00:00  (Alanine Aminotransferase in Serum or Plasma (...   
               2006-02-13 00:00:00                                                NaN   
               2006-05-11 00:00:00                                                NaN   
1000100000369  2013-05-02                                                         NaN   
               2013-12-30                                                         NaN   
...                                                                               ...   
12120120066987 2017-07-11 00:00:00                                                NaN   
               2017-09-25 00:00:00                                                NaN   
               2017-10-02 00:00:00                                                NaN   
               2018-02-02 00:00:00                                                NaN   
               2018-03-20 00:00:00                                                NaN   

                                                                                   

In [133]:
eTests = []
for t in tests :
    for i in range(1,4) :
        eTests.append(t+str(i))

In [134]:
RowFeatures = pd.DataFrame(data = 0,
                       index = PVDF.index,
                       columns=eTests)
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100000266  2006-02-07 00:00:00                                                  0      
               2006-02-13 00:00:00                                                  0      
               2006-05-11 00:00:00                                                  0      
1000100000369  2013-05-02                                                           0      
               2013-12-30                                                           0      
...                                                                               ...      
12120120066987 2017-07-11 00:00:00                                                  0      
               2017-09-25 00:00:00                       

In [136]:
for index, row in RowFeatures.iterrows() :
    temp = PVDF.loc[index].dropna()
    for t in temp :
        test = t[0] + str(int(t[1]))
        row[test] = 1

In [137]:
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100000266  2006-02-07 00:00:00                                                  0    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  0    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100000266  2006-02-07 00:00:00                                                  1    
               2006-02-13 00:00:00                                                  0    
               2006-05-11 00:00:00                                                  1    
1000100000369  2013-05-02                                                           0    
               2013-12-30                                                           0    
...                                                                               ...    
12120120066987 2017-07-11 00:00:00                                                  0    
               2017-09-25 00:00:00                                                  0    
               2017-10-02 00:00:00                                                  0    
               2018-02-02 00:00:00                                                  0    
               2018-03-20 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100000266  2006-02-07 00:00:00                                                  0      
               2006-02-13 00:00:00                                                  0      
               2006-05-11 00:00:00                                                  0      
1000100000369  2013-05-02                                                           0      
               2013-12-30                                                           0      
...                                                                               ...      
12120120066987 2017-07-11 00:00:00                                                  0      
               2017-09-25 00:00:00                       

In [139]:
RowFeatures.iloc[0]

Alanine Aminotransferase in Serum or Plasma (ALT)1      0
Alanine Aminotransferase in Serum or Plasma (ALT)2      0
Alanine Aminotransferase in Serum or Plasma (ALT)3      1
Aspartate Aminotransferase in Serum or Plasma (AST)1    0
Aspartate Aminotransferase in Serum or Plasma (AST)2    0
Aspartate Aminotransferase in Serum or Plasma (AST)3    0
Urea (mM)1                                              0
Urea (mM)2                                              0
Urea (mM)3                                              0
Plasma Creatinine Clearance (eGFR)1                     1
Plasma Creatinine Clearance (eGFR)2                     0
Plasma Creatinine Clearance (eGFR)3                     0
Potassium (mM)1                                         0
Potassium (mM)2                                         0
Potassium (mM)3                                         0
Sodium (mM)1                                            0
Sodium (mM)2                                            0
Sodium (mM)3  

In [140]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 18)
visits_emb = svd.fit_transform(RowFeatures)
reducedDF = pd.DataFrame(data=visits_emb, index=RowFeatures.index)

In [141]:
reducedDF.head()

0         1         2         3   \
1000100000266 2006-02-07 00:00:00  1.868043  0.827565 -0.854393 -0.605995   
              2006-02-13 00:00:00  0.008236 -0.000247  0.008301  0.004933   
              2006-05-11 00:00:00  0.522429 -0.093851 -0.495433 -0.475502   
1000100000369 2013-05-02           0.217349 -0.067416 -0.517735 -0.446745   
              2013-12-30           0.312957  0.012879 -0.173986 -0.248978   

                                         4         5         6         7   \
1000100000266 2006-02-07 00:00:00  0.196777 -0.340435  0.436092  0.840309   
              2006-02-13 00:00:00  0.000224  0.022433  0.009144 -0.004769   
              2006-05-11 00:00:00 -0.031012  0.287704  0.010026  0.116601   
1000100000369 2013-05-02          -0.268812  0.291041 -0.015409  0.089437   
              2013-12-30          -0.239500  0.306567 -0.041255  0.007301   

                                         8         9         10        11  \
1000100000266 2006-02-07 00:00:00 -0.595056 -0.387273 -0.127773 -0.972443   
              2006-02-13 00:00:00 -0.004582 -0.014191  0.002546 -0.006146   
              2006-05-11 00:00:00 -0.168452 -0.399335  0.087036 -0.175728   
1000100000369 2013-05-02          -0.085305 -0.078411 -0.050775 -0.015992   
              2013-12-30          -0.002545  0.362621  0.753217 -0.016815   

                                         12        13        14        15  \
1000100000266 2006-02-07 00:00:00  0.327869  0.406777 -0.021037 -0.765665   
              2006-02-13 00:00:00  0.004930 -0.006533  0.031763 -0.056113   
              2006-05-11 00:00:00  0.591820  0.632585  0.107864 -0.279896   
1000100000369 2013-05-02           0.095255  0.080746  0.054504 -0.013319   
              2013-12-30          -0.064316  0.082247 -0.304340 -0.337367   

                                         16        17  
1000100000266 2006-02-07 00:00:00 -0.219601 -0.412076  
              2006-02-13 00:00:00  0.945934 -0.305841  
              2006-05-11 00:00:00 -0.235219 -0.655685  
1000100000369 2013-05-02           0.017996 -0.003122  
              2013-12-30          -0.046375 -0.043594

In [142]:
IDs = neg_samp

In [143]:
VisitHistorySVD = {}
for index, row in reducedDF.iterrows() :
    temp = int(index[0])
    featureVector = list(row)
    if temp not in VisitHistorySVD.keys():
        if temp in IDs :
            VisitHistorySVD[temp] = [0]
        else :
            VisitHistorySVD[temp] = [1]
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector




In [145]:
VisitHistorySVD[1000100000266]

[1,
 1.8680427551193475,
 0.8275646352235301,
 -0.8543928532024485,
 -0.6059948106181343,
 0.19677712287235566,
 -0.3404354259706436,
 0.4360920226277335,
 0.8403087032337714,
 -0.5950560422880431,
 -0.38727303490804504,
 -0.12777279146170167,
 -0.9724425808084254,
 0.3278686738885745,
 0.40677723692715406,
 -0.021036919016778448,
 -0.7656649619640798,
 -0.2196008943068246,
 -0.41207610548613843,
 0.008235590483780262,
 -0.0002473941236317563,
 0.008301145852266224,
 0.0049329740169191914,
 0.00022364072785285995,
 0.022433079628098027,
 0.00914414365686058,
 -0.004769178361952284,
 -0.004581797404594678,
 -0.014191426616405996,
 0.0025461310326981472,
 -0.006146440995726016,
 0.004929870241389122,
 -0.006532584316185356,
 0.03176348239362354,
 -0.05611311594409088,
 0.9459336622823001,
 -0.3058407934454371,
 0.5224285913360496,
 -0.09385096448160184,
 -0.49543316133510723,
 -0.4755019567771033,
 -0.031012402743893303,
 0.28770406077635147,
 0.010025860076259782,
 0.11660131861659022,


In [146]:
m = [0]
for k,v in VisitHistorySVD.items() :
    temp = len(v)
    if temp > m[-1] :
        m.append(k)
        m.append(temp)
        
print(m)

[0, 1000100000266, 55, 1000100000369, 181, 1000100003823, 595, 1000100034421, 865, 5050080001678, 883]


In [147]:
# Fill in to 900
for k,v in VisitHistorySVD.items() :
    fillLen = 901 - len(v)
    VisitHistorySVD[k] = v + [0]*fillLen

In [148]:
# MODELING
# Convert data to x y
x, y = [], []
for k,v in VisitHistorySVD.items() :
    y.append(v[0])
    x.append(v[1:])

In [149]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [165]:
# Split data
x_train, x_test, y_train, y_test = \
    train_test_split(x, y, test_size = 0.1)
print(len(x_train), 'training')
print(len(x_test), 'testing')

2919 training
325 testing


In [179]:
# Bidirectional GRU

# Random search drop out rate and number of units/neurons
x_train = np.array(x_train)
x_test = np.array(x_test)
input_shape = (x_train.shape[1], 1) #900,1

units = 100


model = Sequential()
model.add(Reshape((50, 18), input_shape=(900,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2 # for regularization
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(LSTM(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(x_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 15,
    validation_split = 0.1
)




Number of hidden units:  100 Dropout:  0.2
Training...
Epoch 1/15
21/21 [==============================] - 1s 52ms/step - loss: 0.6896 - accuracy: 0.5397 - val_loss: 0.6869 - val_accuracy: 0.5685
Epoch 2/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6833 - accuracy: 0.5577 - val_loss: 0.6843 - val_accuracy: 0.5993
Epoch 3/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6802 - accuracy: 0.5797 - val_loss: 0.6800 - val_accuracy: 0.5685
Epoch 4/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6778 - accuracy: 0.5799 - val_loss: 0.6803 - val_accuracy: 0.5890
Epoch 5/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6726 - accuracy: 0.5923 - val_loss: 0.6806 - val_accuracy: 0.5788
Epoch 6/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6662 - accuracy: 0.6009 - val_loss: 0.6807 - val_accuracy: 0.5959
Epoch 7/15
21/21 [==============================] - 1s 27ms/step - loss: 0.6698 - accuracy: 0.5849 

In [180]:
print(model.evaluate(x_train, np.array(y_train)))
preds = model.predict(x_test)

92/92 [==============================] - 1s 8ms/step - loss: 0.7011 - accuracy: 0.5546
[0.7011363506317139, 0.5546420216560364]


In [181]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

0.5339884874649701